In [1]:
import findspark
findspark.init()

In [2]:
# test if pyspark is working. Takes a minute to finish.
import pyspark
import random

sc = pyspark.SparkContext(appName="Pi")
num_samples = 10000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

3.1216


In [3]:
# create a SparkSession.
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
spark    

In [4]:
# reading CSV with custom schema.
from pyspark.sql.types import *

customSchema = StructType([ \
    StructField("account_id", IntegerType(), True), \
    StructField("district_id", IntegerType(), True), \
    StructField("frequency", StringType(), True), \
    StructField("date", IntegerType(), True)])

In [5]:
df = spark.read.format('csv') \
          .option('header', 'true') \
          .option('delimiter', ';') \
          .load('s3://justinngbucket/Data/berka_dataset/account.asc', schema=customSchema)

In [6]:
df.show()

+----------+-----------+------------------+------+
|account_id|district_id|         frequency|  date|
+----------+-----------+------------------+------+
|       576|         55|  POPLATEK MESICNE|930101|
|      3818|         74|  POPLATEK MESICNE|930101|
|       704|         55|  POPLATEK MESICNE|930101|
|      2378|         16|  POPLATEK MESICNE|930101|
|      2632|         24|  POPLATEK MESICNE|930102|
|      1972|         77|  POPLATEK MESICNE|930102|
|      1539|          1|POPLATEK PO OBRATU|930103|
|       793|         47|  POPLATEK MESICNE|930103|
|      2484|         74|  POPLATEK MESICNE|930103|
|      1695|         76|  POPLATEK MESICNE|930103|
|      1726|         48|  POPLATEK MESICNE|930103|
|      2881|         70|  POPLATEK MESICNE|930104|
|      2357|         19|  POPLATEK MESICNE|930104|
|      2177|         62|  POPLATEK MESICNE|930104|
|       485|          6|POPLATEK PO OBRATU|930104|
|       652|         21|  POPLATEK MESICNE|930105|
|      9635|         70|  POPLA

In [7]:
df.printSchema()

root
 |-- account_id: integer (nullable = true)
 |-- district_id: integer (nullable = true)
 |-- frequency: string (nullable = true)
 |-- date: integer (nullable = true)



In [10]:
# retrieve minimum.
df.groupby().min('date').show()

+---------+
|min(date)|
+---------+
|   930101|
+---------+



In [56]:
df.groupby('district_id', 'frequency').min('date').show()

+-----------+------------------+---------+
|district_id|         frequency|min(date)|
+-----------+------------------+---------+
|          1|POPLATEK PO OBRATU|   930103|
|         13|POPLATEK PO OBRATU|   930216|
|         52|  POPLATEK MESICNE|   930121|
|         65|  POPLATEK MESICNE|   930316|
|         57|  POPLATEK MESICNE|   930418|
|         68|    POPLATEK TYDNE|   930510|
|         77|  POPLATEK MESICNE|   930102|
|         31|  POPLATEK MESICNE|   930213|
|         75|  POPLATEK MESICNE|   930124|
|         74|  POPLATEK MESICNE|   930101|
|         30|  POPLATEK MESICNE|   930127|
|         31|    POPLATEK TYDNE|   940522|
|          9|    POPLATEK TYDNE|   960226|
|         69|  POPLATEK MESICNE|   930220|
|         43|    POPLATEK TYDNE|   930526|
|         18|  POPLATEK MESICNE|   930726|
|         27|  POPLATEK MESICNE|   930127|
|          1|  POPLATEK MESICNE|   930108|
|         69|POPLATEK PO OBRATU|   941117|
|         76|    POPLATEK TYDNE|   931223|
+----------

In [11]:
df.registerTempTable("accounts")

In [12]:
spark.sql('select min(date) as min_date, max(date) as max_date from accounts').show()

+--------+--------+
|min_date|max_date|
+--------+--------+
|  930101|  971229|
+--------+--------+



In [13]:
df.describe('date').show()

+-------+------------------+
|summary|              date|
+-------+------------------+
|  count|              4500|
|   mean| 951654.6086666667|
| stddev|14842.188376633594|
|    min|            930101|
|    max|            971229|
+-------+------------------+



In [16]:
df.describe('date').filter("summary = 'stddev'").collect()

[Row(summary='stddev', date='14842.188376633594')]

In [21]:
# retreiving stddev as a float.
float(df.describe('date').filter("summary = 'stddev'").collect()[0].asDict()['date'])

14842.188376633594

In [24]:
# get quantiles.
df.stat.approxQuantile("date", [0.25,0.5,0.75],0.0)

[931227.0, 960102.0, 961101.0]

In [54]:
df.agg({'date': 'max'}).show()

+---------+
|max(date)|
+---------+
|   971229|
+---------+



In [37]:
# save file as parquet.
df.write.parquet('s3://justinngbucket/accounts5.parquet')

In [38]:
# select directly from parquet file.
spark.sql("SELECT * FROM parquet.`s3://justinngbucket/accounts5.parquet`").show()

+----------+-----------+------------------+------+
|account_id|district_id|         frequency|  date|
+----------+-----------+------------------+------+
|       576|         55|  POPLATEK MESICNE|930101|
|      3818|         74|  POPLATEK MESICNE|930101|
|       704|         55|  POPLATEK MESICNE|930101|
|      2378|         16|  POPLATEK MESICNE|930101|
|      2632|         24|  POPLATEK MESICNE|930102|
|      1972|         77|  POPLATEK MESICNE|930102|
|      1539|          1|POPLATEK PO OBRATU|930103|
|       793|         47|  POPLATEK MESICNE|930103|
|      2484|         74|  POPLATEK MESICNE|930103|
|      1695|         76|  POPLATEK MESICNE|930103|
|      1726|         48|  POPLATEK MESICNE|930103|
|      2881|         70|  POPLATEK MESICNE|930104|
|      2357|         19|  POPLATEK MESICNE|930104|
|      2177|         62|  POPLATEK MESICNE|930104|
|       485|          6|POPLATEK PO OBRATU|930104|
|       652|         21|  POPLATEK MESICNE|930105|
|      9635|         70|  POPLA

In [41]:
# caching into memory. 
# cache -> persist(StorageLevel.MEMORY_ONLY)
df.cache()

DataFrame[account_id: int, district_id: int, frequency: string, date: int]

In [47]:
# persist using memory and disk.
from pyspark import StorageLevel
df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[account_id: int, district_id: int, frequency: string, date: int]

In [44]:
df.unpersist()

DataFrame[account_id: int, district_id: int, frequency: string, date: int]

In [50]:
df.account_id

Column<b'account_id'>

In [57]:
df.corr('district_id', 'date')

-0.017291702248228277

In [58]:
df.count()

4500

In [61]:
spark.sql('select count(*) as num_rows from accounts').show()

+--------+
|num_rows|
+--------+
|    4500|
+--------+



In [63]:
df.crosstab('district_id', 'frequency').show()

+---------------------+----------------+------------------+--------------+
|district_id_frequency|POPLATEK MESICNE|POPLATEK PO OBRATU|POPLATEK TYDNE|
+---------------------+----------------+------------------+--------------+
|                   69|              43|                 1|             4|
|                    5|              60|                 0|             5|
|                   10|              46|                 1|             1|
|                   56|              40|                 2|             2|
|                   42|              45|                 2|             1|
|                   24|              39|                 2|             1|
|                   37|              34|                 1|             2|
|                   25|              40|                 0|             2|
|                   52|              50|                 4|             3|
|                   14|              37|                 1|             3|
|                   20|  

In [67]:
df.cube('district_id', 'frequency').count().orderBy("district_id", "frequency").show()

+-----------+------------------+-----+
|district_id|         frequency|count|
+-----------+------------------+-----+
|       null|              null| 4500|
|       null|  POPLATEK MESICNE| 4167|
|       null|POPLATEK PO OBRATU|   93|
|       null|    POPLATEK TYDNE|  240|
|          1|              null|  554|
|          1|  POPLATEK MESICNE|  509|
|          1|POPLATEK PO OBRATU|    9|
|          1|    POPLATEK TYDNE|   36|
|          2|              null|   42|
|          2|  POPLATEK MESICNE|   37|
|          2|POPLATEK PO OBRATU|    2|
|          2|    POPLATEK TYDNE|    3|
|          3|              null|   50|
|          3|  POPLATEK MESICNE|   47|
|          3|    POPLATEK TYDNE|    3|
|          4|              null|   48|
|          4|  POPLATEK MESICNE|   44|
|          4|POPLATEK PO OBRATU|    2|
|          4|    POPLATEK TYDNE|    2|
|          5|              null|   65|
+-----------+------------------+-----+
only showing top 20 rows



In [71]:
spark.sql('select count(*) from accounts where frequency is null').show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [80]:
df.select('account_id', 'frequency').distinct().show()

+----------+----------------+
|account_id|       frequency|
+----------+----------------+
|      3818|POPLATEK MESICNE|
|      2648|POPLATEK MESICNE|
|      1337|POPLATEK MESICNE|
|      8759|POPLATEK MESICNE|
|       198|POPLATEK MESICNE|
|      3479|POPLATEK MESICNE|
|      5735|POPLATEK MESICNE|
|      3297|POPLATEK MESICNE|
|       720|POPLATEK MESICNE|
|      8327|  POPLATEK TYDNE|
|      2674|POPLATEK MESICNE|
|      1581|POPLATEK MESICNE|
|      3389|POPLATEK MESICNE|
|       631|POPLATEK MESICNE|
|      3015|POPLATEK MESICNE|
|      3848|POPLATEK MESICNE|
|      5250|  POPLATEK TYDNE|
|      1500|POPLATEK MESICNE|
|      3043|POPLATEK MESICNE|
|      3262|POPLATEK MESICNE|
+----------+----------------+
only showing top 20 rows



In [83]:
df.dropna(how='any').count()

4500

In [82]:
df.dtypes

[('account_id', 'int'),
 ('district_id', 'int'),
 ('frequency', 'string'),
 ('date', 'int')]

In [84]:
df.fillna(50, ['account_id', 'district_id'])

DataFrame[account_id: int, district_id: int, frequency: string, date: int]

In [88]:
df.filter("date = 930101").show()

+----------+-----------+----------------+------+
|account_id|district_id|       frequency|  date|
+----------+-----------+----------------+------+
|       576|         55|POPLATEK MESICNE|930101|
|      3818|         74|POPLATEK MESICNE|930101|
|       704|         55|POPLATEK MESICNE|930101|
|      2378|         16|POPLATEK MESICNE|930101|
+----------+-----------+----------------+------+



In [90]:
df.filter(df['date'] == 930101).show()

+----------+-----------+----------------+------+
|account_id|district_id|       frequency|  date|
+----------+-----------+----------------+------+
|       576|         55|POPLATEK MESICNE|930101|
|      3818|         74|POPLATEK MESICNE|930101|
|       704|         55|POPLATEK MESICNE|930101|
|      2378|         16|POPLATEK MESICNE|930101|
+----------+-----------+----------------+------+



In [95]:
# this doesn't seem to work.
def print_acc(acc):
    print(acc.account_id)
df.filter(df['date'] == 930101).foreach(print_acc)

In [101]:
# getting the frequent items (possible false positives)
df.freqItems(['date'], support=None).show()

+--------------------+
|      date_freqItems|
+--------------------+
|[971114, 970504, ...|
+--------------------+



In [102]:
df_trans = spark.read.format('csv') \
          .option('header', 'true') \
          .option('delimiter', ';') \
          .option('inferschema', 'true') \
          .load('s3://justinngbucket/Data/berka_dataset/trans.asc')

In [103]:
df_trans.dtypes

[('trans_id', 'int'),
 ('account_id', 'int'),
 ('date', 'int'),
 ('type', 'string'),
 ('operation', 'string'),
 ('amount', 'double'),
 ('balance', 'double'),
 ('k_symbol', 'string'),
 ('bank', 'string'),
 ('account', 'int')]

In [105]:
df_trans.show()

+--------+----------+------+------+-------------+------+-------+--------+----+--------+
|trans_id|account_id|  date|  type|    operation|amount|balance|k_symbol|bank| account|
+--------+----------+------+------+-------------+------+-------+--------+----+--------+
|  695247|      2378|930101|PRIJEM|        VKLAD| 700.0|  700.0|    null|null|    null|
|  171812|       576|930101|PRIJEM|        VKLAD| 900.0|  900.0|    null|null|    null|
|  207264|       704|930101|PRIJEM|        VKLAD|1000.0| 1000.0|    null|null|    null|
| 1117247|      3818|930101|PRIJEM|        VKLAD| 600.0|  600.0|    null|null|    null|
|  579373|      1972|930102|PRIJEM|        VKLAD| 400.0|  400.0|    null|null|    null|
|  771035|      2632|930102|PRIJEM|        VKLAD|1100.0| 1100.0|    null|null|    null|
|  452728|      1539|930103|PRIJEM|        VKLAD| 600.0|  600.0|    null|null|    null|
|  725751|      2484|930103|PRIJEM|        VKLAD|1100.0| 1100.0|    null|null|    null|
|  497211|      1695|930103|PRIJ

In [116]:
df2 = df.join(df_trans, ['account_id'], 'inner')
df2.show()

+----------+-----------+------------------+------+--------+------+------+-------------+------+-------+--------+----+--------+
|account_id|district_id|         frequency|  date|trans_id|  date|  type|    operation|amount|balance|k_symbol|bank| account|
+----------+-----------+------------------+------+--------+------+------+-------------+------+-------+--------+----+--------+
|      2378|         16|  POPLATEK MESICNE|930101|  695247|930101|PRIJEM|        VKLAD| 700.0|  700.0|    null|null|    null|
|       576|         55|  POPLATEK MESICNE|930101|  171812|930101|PRIJEM|        VKLAD| 900.0|  900.0|    null|null|    null|
|       704|         55|  POPLATEK MESICNE|930101|  207264|930101|PRIJEM|        VKLAD|1000.0| 1000.0|    null|null|    null|
|      3818|         74|  POPLATEK MESICNE|930101| 1117247|930101|PRIJEM|        VKLAD| 600.0|  600.0|    null|null|    null|
|      1972|         77|  POPLATEK MESICNE|930102|  579373|930102|PRIJEM|        VKLAD| 400.0|  400.0|    null|null|  

In [110]:
df2.count()

1056320

In [111]:
df_trans.count()

1056320

In [112]:
df_trans.registerTempTable("trans")

In [133]:
# sql way of joining.
df2 = spark.sql("select acc.district_id, acc.frequency, t.* from accounts acc inner join trans t on acc.account_id = t.account_id")
df2.show()

+-----------+------------------+--------+----------+------+------+-------------+------+-------+--------+----+--------+
|district_id|         frequency|trans_id|account_id|  date|  type|    operation|amount|balance|k_symbol|bank| account|
+-----------+------------------+--------+----------+------+------+-------------+------+-------+--------+----+--------+
|         16|  POPLATEK MESICNE|  695247|      2378|930101|PRIJEM|        VKLAD| 700.0|  700.0|    null|null|    null|
|         55|  POPLATEK MESICNE|  171812|       576|930101|PRIJEM|        VKLAD| 900.0|  900.0|    null|null|    null|
|         55|  POPLATEK MESICNE|  207264|       704|930101|PRIJEM|        VKLAD|1000.0| 1000.0|    null|null|    null|
|         74|  POPLATEK MESICNE| 1117247|      3818|930101|PRIJEM|        VKLAD| 600.0|  600.0|    null|null|    null|
|         77|  POPLATEK MESICNE|  579373|      1972|930102|PRIJEM|        VKLAD| 400.0|  400.0|    null|null|    null|
|         24|  POPLATEK MESICNE|  771035|      2

In [134]:
splits = df2.randomSplit([0.5, 0.5], 24)

In [135]:
splits[0].count()

528267

In [136]:
splits[1].count()

528053

In [137]:
df2.rdd.getNumPartitions()

4

In [138]:
# get distinct types.
df2.select(df2['type']).distinct().show()

+------+
|  type|
+------+
| VYDAJ|
|PRIJEM|
| VYBER|
+------+



In [140]:
# translating type to English.
replace_dict = {'PRIJEM':'CREDIT', 'VYDAJ':'WITHDRAWAL', 'VYBER':'WITHDRAWAL'}
df2.replace(replace_dict, subset=['type']).show()


+-----------+------------------+--------+----------+------+------+-------------+------+-------+--------+----+--------+
|district_id|         frequency|trans_id|account_id|  date|  type|    operation|amount|balance|k_symbol|bank| account|
+-----------+------------------+--------+----------+------+------+-------------+------+-------+--------+----+--------+
|         16|  POPLATEK MESICNE|  695247|      2378|930101|CREDIT|        VKLAD| 700.0|  700.0|    null|null|    null|
|         55|  POPLATEK MESICNE|  171812|       576|930101|CREDIT|        VKLAD| 900.0|  900.0|    null|null|    null|
|         55|  POPLATEK MESICNE|  207264|       704|930101|CREDIT|        VKLAD|1000.0| 1000.0|    null|null|    null|
|         74|  POPLATEK MESICNE| 1117247|      3818|930101|CREDIT|        VKLAD| 600.0|  600.0|    null|null|    null|
|         77|  POPLATEK MESICNE|  579373|      1972|930102|CREDIT|        VKLAD| 400.0|  400.0|    null|null|    null|
|         24|  POPLATEK MESICNE|  771035|      2

In [141]:
# translating operation to English.
df2.select('operation').distinct().show()

+--------------+
|     operation|
+--------------+
|         VKLAD|
|          null|
| PREVOD Z UCTU|
|  VYBER KARTOU|
|PREVOD NA UCET|
|         VYBER|
+--------------+



In [142]:
replace_dict = {'VYBER KARTOU':'CC_WITHDRAWAL', 
                'VKLAD':'CREDIT_IN_CASH', 
                'PREVOD Z UCTU':'COLLECTION_FROM_OTHER_BANK', 
                'VYBER':'WITHDRAWAL_IN_CASH',
                'PREVOD NA UCET':'REMITTANCE_TO_OTHER_BANK'}
df2.replace(replace_dict, subset='operation').na.fill('UNKNOWN', subset='operation').show()

+-----------+------------------+--------+----------+------+------+--------------------+------+-------+--------+----+--------+
|district_id|         frequency|trans_id|account_id|  date|  type|           operation|amount|balance|k_symbol|bank| account|
+-----------+------------------+--------+----------+------+------+--------------------+------+-------+--------+----+--------+
|         16|  POPLATEK MESICNE|  695247|      2378|930101|PRIJEM|      CREDIT_IN_CASH| 700.0|  700.0|    null|null|    null|
|         55|  POPLATEK MESICNE|  171812|       576|930101|PRIJEM|      CREDIT_IN_CASH| 900.0|  900.0|    null|null|    null|
|         55|  POPLATEK MESICNE|  207264|       704|930101|PRIJEM|      CREDIT_IN_CASH|1000.0| 1000.0|    null|null|    null|
|         74|  POPLATEK MESICNE| 1117247|      3818|930101|PRIJEM|      CREDIT_IN_CASH| 600.0|  600.0|    null|null|    null|
|         77|  POPLATEK MESICNE|  579373|      1972|930102|PRIJEM|      CREDIT_IN_CASH| 400.0|  400.0|    null|null|  

In [147]:
# literals.
from pyspark.sql.functions import lit

df2.select('account_id', 'trans_id', 'district_id').withColumn('NULL_COLUMN', lit(None).cast(StringType())).show()

+----------+--------+-----------+-----------+
|account_id|trans_id|district_id|NULL_COLUMN|
+----------+--------+-----------+-----------+
|       576| 3549684|         55|       null|
|       576|  171974|         55|       null|
|       576|  171956|         55|       null|
|       576|  171884|         55|       null|
|       576|  172120|         55|       null|
|       576|  172107|         55|       null|
|       576| 3549683|         55|       null|
|       576|  171955|         55|       null|
|       576|  171883|         55|       null|
|       576|  172119|         55|       null|
|       576|  172106|         55|       null|
|       576| 3549682|         55|       null|
|       576|  171954|         55|       null|
|       576|  171882|         55|       null|
|       576|  172105|         55|       null|
|       576| 3549681|         55|       null|
|       576|  171953|         55|       null|
|       576|  171881|         55|       null|
|       576|  172104|         55| 